In [3]:
!pip install azureml-core

     |████████████████████████████████| 1.5MB 74kB/s  eta 0:00:01
     |████████████████████████████████| 92kB 730kB/s eta 0:00:011
     |████████████████████████████████| 112kB 5.0MB/s eta 0:00:01
     |████████████████████████████████| 153kB 5.8MB/s eta 0:00:01
     |████████████████████████████████| 40kB 277kB/s eta 0:00:01
     |████████████████████████████████| 552kB 1.7MB/s eta 0:00:01
     |████████████████████████████████| 8.6MB 13kB/s  eta 0:00:01     |██▍                             | 645kB 1.1MB/s eta 0:00:08     |████                            | 1.1MB 65kB/s eta 0:01:56     |████▌                           | 1.2MB 65kB/s eta 0:01:54     |████▋                           | 1.2MB 179kB/s eta 0:00:41     |███████████████████▍            | 5.2MB 94kB/s eta 0:00:36     |████████████████████▎           | 5.4MB 94kB/s eta 0:00:33     |█████████████████████▋          | 5.8MB 94kB/s eta 0:00:30     |████████████████████████▎       | 6.5MB 94kB/s eta 0:00:22
     |███████████████████

  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
     |████████████████████████████████| 409kB 8.5MB/s eta 0:00:01
  Found existing installation: requests 2.14.2
    Uninstalling requests-2.14.2:
      Successfully uninstalled requests-2.14.2
  Found existing installation: cffi 1.7.0
    Uninstalling cffi-1.7.0:
      Successfully uninstalled cffi-1.7.0
  Found existing installation: cryptography 2.0.3
    Uninstalling cryptography-2.0.3:
      Successfully uninstalled cryptography-2.0.3
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 658kB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [6]:
from azureml.core import Workspace
ws_new = Workspace.from_config(path="./.azureml/fc_config.json")
ws_new.get_details()

{'applicationInsights': '/subscriptions/53dc328a-1da3-4f32-a012-85cebcd27290/resourcegroups/arpita/providers/microsoft.insights/components/frauddetinsights049c8a8f',
 'creationTime': '2020-06-22T16:35:28.0152590+00:00',
 'description': '',
 'discoveryUrl': 'https://eastus2.experiments.azureml.net/discovery',
 'friendlyName': 'fraud_detect',
 'hbiWorkspace': False,
 'id': '/subscriptions/53dc328a-1da3-4f32-a012-85cebcd27290/resourceGroups/Arpita/providers/Microsoft.MachineLearningServices/workspaces/fraud_detect',
 'identityPrincipalId': 'c8142116-3151-4484-a277-1a0f9ddb9b1f',
 'identityTenantId': '652b1a9f-474c-4307-8c46-8dbb757f11b3',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/53dc328a-1da3-4f32-a012-85cebcd27290/resourcegroups/arpita/providers/microsoft.keyvault/vaults/frauddetkeyvault0519236d',
 'location': 'eastus2',
 'name': 'fraud_detect',
 'sku': 'Basic',
 'storageAccount': '/subscriptions/53dc328a-1da3-4f32-a012-85cebcd27290/resourcegroups/arpita/providers/

In [8]:
!pip install joblib

     |████████████████████████████████| 294 kB 1.8 MB/s eta 0:00:01


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from azureml.core import Run
import joblib
from sklearn.linear_model import LogisticRegression


In [10]:
df = pd.read_csv('credit_card_data.csv')

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['isForeignTransaction'] = le.fit_transform(df['isForeignTransaction'])
df['Is declined'] = le.fit_transform(df['Is declined'])
df['isHighRiskCountry'] = le.fit_transform(df['isHighRiskCountry'])
df['isFradulent'] = le.fit_transform(df['isFradulent'])

In [12]:
X = df.drop(['isFradulent'],axis=1)
y = df['isFradulent']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, y_train.shape,X_test.shape,y_test.shape

((2460, 10), (2460,), (615, 10), (615,))

In [13]:
X_train.columns

Index(['Merchant_id', 'Average Amount/transaction/day', 'Transaction_amount',
       'Is declined', 'Total Number of declines/day', 'isForeignTransaction',
       'isHighRiskCountry', 'Daily_chargeback_avg_amt', '6_month_avg_chbk_amt',
       '6-month_chbk_freq'],
      dtype='object')

In [14]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=123).fit(X_train, y_train)
clf.predict(X_test)

clf.score(X_test, y_test)

0.832520325203252

### save the model

In [15]:
# Save the trained model
joblib.dump(value=clf, filename='model_fd1.pkl')

['model_fd1.pkl']

### register the model

In [16]:
from azureml.core.model import Model
model = Model.register(workspace=ws_new, model_path="./model_fd1.pkl", model_name="frauddetection")

Registering model frauddetection


In [18]:
from azureml.core.model import Model
print(Model.list(workspace = ws_new))

[Model(workspace=Workspace.create(name='fraud_detect', subscription_id='53dc328a-1da3-4f32-a012-85cebcd27290', resource_group='Arpita'), name=frauddetection, id=frauddetection:1, version=1, tags={}, properties={})]


In [28]:
#print(Model.get_model_path('frauddetection'))
Model.get_model_path('frauddetection', version=1, _workspace=ws_new)

'azureml-models/frauddetection/1/model_fd1.pkl'

In [29]:
model = Model(ws_new,'frauddetection')
print(model)

Model(workspace=Workspace.create(name='fraud_detect', subscription_id='53dc328a-1da3-4f32-a012-85cebcd27290', resource_group='Arpita'), name=frauddetection, id=frauddetection:1, version=1, tags={}, properties={})


In [30]:
pd.__version__

'0.19.2'

In [31]:
np.__version__

'1.17.3'

In [33]:
import sys

print(sys.version)

3.5.4 |Anaconda custom (64-bit)| (default, Oct 13 2017, 11:22:58) 
[GCC 7.2.0]


In [34]:
import sklearn

In [35]:
sklearn.__version__

'0.19.1'

In [36]:
import platform
print(platform.python_version())

3.5.4


In [37]:
import joblib
joblib.__version__

'0.14.1'